In [ ]:
!pip install easyocr

In [ ]:
import easyocr
import cv2
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
import pickle
import time
import concurrent.futures

In [ ]:
img_path = '/content/plat nomor.jpg'
img_path2 = '/content/plat nomor 2.jpg'

In [ ]:
import cv2
import easyocr

# Mendefinisikan path gambar
img_path = '/content/plat nomor.jpg'
img_path2 = '/content/plat nomor 2.jpg'

# Membaca teks dari gambar
reader = easyocr.Reader(['en'])
result = reader.readtext(img_path)


# Melakukan serialisasi objek result menggunakan pickle
serialized_result = pickle.dumps(result)

# Menyimpan hasil serialisasi ke file
with open('serialized_result.pkl', 'wb') as f:
    f.write(serialized_result)

In [ ]:
# Menampilkan gambar menggunakan OpenCV
img = cv2.imread(img_path)
plt.imshow(img)
plt.show()

In [ ]:
# Membuka file serialized_result.pkl
with open('serialized_result.pkl', 'rb') as f:
    serialized_result = f.read()

# Melakukan deserialisasi objek menggunakan pickle
result = pickle.loads(serialized_result)

# Menggunakan objek yang telah di-deserialisasi
for item in result:
    print(item)

In [ ]:
# Membaca teks dari gambar
reader = easyocr.Reader(['en'])
result2 = reader.readtext(img_path2)

# Melakukan serialisasi objek result menggunakan pickle
serialized_result2 = pickle.dumps(result2)

# Menyimpan hasil serialisasi ke file
with open('serialized_result2.pkl', 'wb') as f:
    f.write(serialized_result2)

In [ ]:
# Membuka file serialized_result.pkl
with open('serialized_result2.pkl', 'rb') as f:
    serialized_result2 = f.read()

# Melakukan deserialisasi objek menggunakan pickle
result2 = pickle.loads(serialized_result2)

# Menggunakan objek yang telah di-deserialisasi
for item in result2:
    print(item)

In [ ]:
img2 = cv2.imread(img_path2)
plt.imshow(img2)
plt.show()

In [ ]:
for i in range(len(result2)):
  top_left = tuple(result2[i][0][0])
  bottom_right = tuple(result2[i][0][2])
  img = cv2.rectangle(img2,top_left,bottom_right,(255,0,0),3)

plt.imshow(img2)
plt.show()

In [ ]:
import cv2
import easyocr

# Baca gambar
image = img_path2

# Pra-pemrosesan gambar jika diperlukan

# Inisialisasi OCR
reader = easyocr.Reader(['en'])

# Gunakan OCR untuk membaca teks
result2 = reader.readtext(image)

# Filter hasil OCR untuk mendapatkan hanya plat nomor
filtered_result2 = [item[1] for item in result2]

# Cetak hasil
for plat_nomor in filtered_result2:
    print(plat_nomor)

# MULTIPROCESSING & SERIAL

In [ ]:
from tqdm import tqdm

# Definisikan gambarnya kembali
img_path = 'plat nomor.jpg'
img_path2 = 'plat nomor 2.jpg'

# Function to process the image and return the OCR result
def process_image(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    return result

# Menjadikan gambarnya menjadi satu path
image_paths = [img_path, img_path2]

# Jumlah processor yang digunakan
num_processors = [1, 2, 4, 8, 16, 32]

total_elapsed_time = 0
serial_execution_time = 0
speedup_scores_amdahl = []
speedup_scores_gustafson = []
speedup_scores_sun_ni = []

for num_procs in num_processors:
    # Menginisiasikan Pool agar dapat memanipulasi proses dengan Multiprocessing
    pool = multiprocessing.Pool(processes=num_procs)

    # Waktu proses dimulai
    start_time = time.time()

    # Proseskan gambar secara Pararel dengan tambahan tqdm
    results = list(tqdm(pool.imap(process_image, image_paths), total=len(image_paths), desc=f"Jalan dengan {num_procs} Prosesor"))

    # Waktu proses berakhir
    end_time = time.time()

    # Menghitung waktu proses
    elapsed_time = end_time - start_time
    total_elapsed_time += elapsed_time

    # Menutup proses
    pool.close()

    # Menyimpan hasil dari pemrosesan OCR dengan serial "Pickel" (SERIAL)
    for i, result in enumerate(results):
        serialized_result = pickle.dumps(result)
        file_name = f'serialized_result_{i+1}_procs_{num_procs}.pkl'
        with open(file_name, 'wb') as f:
            f.write(serialized_result)

    # Menampilkan hasil dari Waktu proses dan SpeedUp dari program diatas
    print(f"")
    print(f"")
    tqdm.write(f"Waktu yang digunakan: {elapsed_time} detik")
    serial_execution_time = elapsed_time / num_procs  # Serial execution time for single processor
    parallel_execution_time = elapsed_time - serial_execution_time  # Parallel execution time for multiple processors
    p = serial_execution_time / elapsed_time  # Proporsi bagian yang tidak dapat di-paralelkan
    speedup_amdahl = 1 / (1 - p + (p / num_procs))  # Speedup score using Amdahl's Law
    speedup_gustafson = num_procs + (1 - p) * num_procs  # Speedup score using Gustafson's Law
    speedup_sun_ni = serial_execution_time / (serial_execution_time + (parallel_execution_time / num_procs))  # Speedup score using Sun and Ni's Law
    tqdm.write(f"Hasil Skor Speed Up dengan teori amdahl: {speedup_amdahl}")
    tqdm.write(f"Hasil Skor Speed Up dengan teori gustafson: {speedup_gustafson}")
    tqdm.write(f"Hasil Skor Speed Up dengan teori sun & ni: {speedup_sun_ni}")
    speedup_scores_amdahl.append(speedup_amdahl)
    speedup_scores_sun_ni.append(speedup_sun_ni)
    speedup_scores_gustafson.append(speedup_gustafson)

# Menampilkan total waktu menjalankan semua proses pada semua processor
tqdm.write(f"Total waktu menjalankan semua proses: {total_elapsed_time} detik")

In [ ]:
# Plotting the speedup values
plt.plot(num_processors, speedup_scores_amdahl, marker='o')
plt.xlabel('Number of Processors')
plt.ylabel('Speedup (seconds per image)')
plt.title('Hasil Algoritma Speedup berdasarkan jumlah processor menggunakan teori Amdahl')
plt.grid(True)
plt.show()

In [ ]:
# Plotting the speedup values
plt.plot(num_processors, speedup_scores_gustafson, marker='o')
plt.xlabel('Number of Processors')
plt.ylabel('Speedup (seconds per image)')
plt.title('Hasil Algoritma Speedup berdasarkan jumlah processor menggunakan teori Gustafson')
plt.grid(True)
plt.show()

In [ ]:
# Plotting the speedup values
plt.plot(num_processors, speedup_scores_gustafson, marker='o')
plt.xlabel('Number of Processors')
plt.ylabel('Speedup (seconds per image)')
plt.title('Hasil Algoritma Speedup berdasarkan jumlah processor menggunakan teori Sun&Ni')
plt.grid(True)
plt.show()

In [ ]:
from tqdm import tqdm

num_processors = [1, 2, 4, 8, 16, 32]

for num_procs in num_processors:
    for i in range(len(image_paths)):
        file_name = f'serialized_result_{i+1}_procs_{num_procs}.pkl'
        with open(file_name, 'rb') as f:
            serialized_result = f.read()

        result = pickle.loads(serialized_result)

        print(f"Hasil Serialisasi untuk file {file_name}:")
        for item in tqdm(result, desc=f"Loading results for {file_name}"):
            print(item)
        print()

# MULTITHREADING

In [ ]:
from tqdm import tqdm
import concurrent.futures

# Definisikan gambarnya kembali
img_path = 'plat nomor.jpg'
img_path2 = 'plat nomor 2.jpg'

# Function to process the image and return the OCR result
def process_image(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    return result

# Menjadikan gambarnya menjadi satu path
image_paths = [img_path, img_path2]

# Jumlah thread yang digunakan
num_threads = [1, 2, 4, 8, 16, 32]

total_elapsed_time = 0
serial_execution_time = 0
speedup_scores_amdahl = []
speedup_scores_gustafson = []
speedup_scores_sun_ni = []

for num_threads in num_threads:
    # Menginisiasikan ThreadPoolExecutor dengan jumlah thread
    executor = concurrent.futures.ThreadPoolExecutor(max_workers=num_threads)

    # Waktu proses dimulai
    start_time = time.time()

    # Melakukan proses dari gambar dengan multithreading
    futures = [executor.submit(process_image, image_path) for image_path in image_paths]

    # Menunggu hingga semua proses selesai
    concurrent.futures.wait(futures)

    # Mengumpulkan hasil dari setiap future
    results = [future.result() for future in futures]

    # Waktu proses berakhir
    end_time = time.time()

    # Menghitung waktu proses
    elapsed_time = end_time - start_time
    total_elapsed_time += elapsed_time

    # Menampilkan hasil dari Waktu proses dan SpeedUp dari program diatas
    print(f"")
    print(f"")
    tqdm.write(f"Jumlah Thread yg dijalankan: {num_threads} thread(s)")
    tqdm.write(f"Waktu yang digunakan: {elapsed_time} detik")
    serial_execution_time = elapsed_time / num_threads  # Serial execution time for single thread
    parallel_execution_time = elapsed_time - serial_execution_time  # Parallel execution time for multiple threads
    p = serial_execution_time / elapsed_time  # Proporsi bagian yang tidak dapat di-paralelkan
    speedup_amdahl = 1 / (1 - p + (p / num_threads))  # Speedup score using Amdahl's Law
    speedup_gustafson = num_threads - p * (num_threads - 1)  # Speedup score using Gustafson's Law
    speedup_sun_ni = serial_execution_time / (serial_execution_time + (parallel_execution_time / num_threads))  # Speedup score using Sun and Ni's Law
    tqdm.write(f"Hasil Skor Speed Up dengan teori amdahl: {speedup_amdahl}")
    tqdm.write(f"Hasil Skor Speed Up dengan teori gustafson: {speedup_gustafson}")
    tqdm.write(f"Hasil Skor Speed Up dengan teori sun & ni: {speedup_sun_ni}")
    speedup_scores_amdahl.append(speedup_amdahl)
    speedup_scores_sun_ni.append(speedup_sun_ni)
    speedup_scores_gustafson.append(speedup_gustafson)

# Menampilkan total waktu menjalankan semua proses pada semua thread
tqdm.write(f"Total waktu menjalankan semua proses: {total_elapsed_time} detik")


In [ ]:
# Plotting the speedup values
plt.plot(num_processors, speedup_scores_amdahl, marker='o')
plt.xlabel('Number of Processors')
plt.ylabel('Speedup (seconds per image)')
plt.title('Hasil Algoritma Speedup berdasarkan jumlah processor menggunakan teori Amdahl')
plt.grid(True)
plt.show()

In [ ]:
# Plotting the speedup values
plt.plot(num_processors, speedup_scores_gustafson, marker='o')
plt.xlabel('Number of Processors')
plt.ylabel('Speedup (seconds per image)')
plt.title('Hasil Algoritma Speedup berdasarkan jumlah processor menggunakan teori gustafson')
plt.grid(True)
plt.show()

In [ ]:
# Plotting the speedup values
plt.plot(num_processors, speedup_scores_sun_ni, marker='o')
plt.xlabel('Number of Processors')
plt.ylabel('Speedup (seconds per image)')
plt.title('Hasil Algoritma Speedup berdasarkan jumlah processor menggunakan teori sun&ni')
plt.grid(True)
plt.show()